<a href="https://colab.research.google.com/github/antonypamo/SavantEngine-RRF-Made/blob/main/SavantEngine_RRF_Clean_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SavantEngine RRF – Colab Runner (Clean)

Este notebook:

1. Clona y instala el repositorio `ProSavantEngine`.
2. Configura opcionalmente Google Drive como fuente de datos RRF.
3. Instancia `SavantEngine`.
4. Ejecuta un demo con los 4 modos:
   - Resonance
   - Φ-node (node)
   - Equation
   - Chat

El objetivo es tener un flujo mínimo y estable para usar el motor SavantEngine.


In [1]:
# @title Environment info
import sys, platform

print("Python version:", sys.version)
print("Platform:", platform.platform())
try:
    import torch
    print("Torch:", torch.__version__)
    print("CUDA available:", torch.cuda.is_available())
except Exception as e:
    print("Torch not available or minimal install:", e)


Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Platform: Linux-6.6.105+-x86_64-with-glibc2.35
Torch: 2.8.0+cu126
CUDA available: False


In [2]:
# @title Clone & install ProSavantEngine
import os

# Trabajamos siempre en /content
os.chdir("/content")
print("Current working dir:", os.getcwd())

repo_url = "https://github.com/antonypamo/ProSavantEngine.git"
repo_path = "/content/ProSavantEngine"

if not os.path.exists(repo_path):
    !git clone {repo_url} {repo_path}
else:
    print("Repo already exists at", repo_path)
    %cd {repo_path}
    !git fetch origin
    # Si quieres forzar reset al main remoto, descomenta:
    # !git reset --hard origin/main

%cd {repo_path}
!python -m pip install -U pip
!pip install -e .


Current working dir: /content
Cloning into '/content/ProSavantEngine'...
remote: Enumerating objects: 228, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 228 (delta 21), reused 0 (delta 0), pack-reused 183 (from 1)
Receiving objects: 100% (228/228), 225.11 KiB | 18.76 MiB/s, done.
Resolving deltas: 100% (107/107), done.
Error downloading object: checkpoint-7500/model.safetensors (a2bf5ef): Smudge error: Error downloading checkpoint-7500/model.safetensors (a2bf5efaffc5881b293cd562b32fcbbff88eb4f87a27492838d86ed533c6fcde): [a2bf5efaffc5881b293cd562b32fcbbff88eb4f87a27492838d86ed533c6fcde] Object does not exist on the server: [404] Object does not exist on the server

Errors logged to /content/ProSavantEngine/.git/lfs/logs/20251115T005335.000771906.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal: checkpoint-7500/model.safetensors: smudge filter lfs failed
You can i

In [ ]:
# @title (Optional) Git LFS pull for large checkpoints
USE_GIT_LFS = True  # cámbialo a False si no lo necesitas

if USE_GIT_LFS:
    %cd /content/ProSavantEngine
    try:
        !apt-get update -y
        !apt-get install -y git-lfs
        !git lfs install
        !git lfs pull
        !pip install -e .
        print("Git LFS initialized and large files pulled.")
    except Exception as e:
        print("Warning: Git LFS setup failed:", e)
else:
    print("Skipping Git LFS setup.")


In [ ]:
# @title (Optional) Mount Google Drive (for RRF data)
MOUNT_DRIVE = True  # pon False si no quieres usar Drive

if MOUNT_DRIVE:
    from google.colab import drive
    drive.mount("/content/drive")
    print("Drive mounted at /content/drive")
else:
    print("Skipping Google Drive mount.")


In [ ]:
# @title Configure RRF_DATA_ROOT
import os

# Candidatos típicos que tú usas; puedes editar esta lista
candidate_dirs = [
    "/content/drive/MyDrive/savant_rrf1",
    "/content/drive/MyDrive/SAVANT_CORE",
    "/content/drive/MyDrive/SavantRRF",
    "/content/drive/MyDrive/savant_rrf_data",
]

chosen = None
for d in candidate_dirs:
    if os.path.isdir(d):
        chosen = d
        break

if chosen is not None:
    os.environ["RRF_DATA_ROOT"] = chosen
    print("RRF_DATA_ROOT set to:", chosen)
else:
    # Si no encuentra carpeta, simplemente lo deja sin configurar.
    # El motor intentará usar rutas por defecto internas.
    print("No RRF data directory found in candidates.")
    print("You can manually set it, e.g.:")
    print('os.environ["RRF_DATA_ROOT"] = "/content/drive/MyDrive/savant_rrf1"')


In [ ]:
# Ejemplo manual para fijar el RRF_DATA_ROOT (modifica la ruta según tu estructura)
# import os
# os.environ["RRF_DATA_ROOT"] = "/content/drive/MyDrive/savant_rrf1"
# print("Manual RRF_DATA_ROOT:", os.environ["RRF_DATA_ROOT"])


In [ ]:
# @title Import SavantEngine
from importlib import import_module

try:
    module = import_module("prosavant_engine.savant_engine")
    SavantEngine = getattr(module, "SavantEngine")
    print("SavantEngine imported successfully from prosavant_engine.savant_engine")
except Exception as e:
    print("Error importing SavantEngine:", e)
    raise

engine = SavantEngine()
print("SavantEngine instance created.")


In [ ]:
# @title Run SavantEngine demo (4 modes)

def run_savant_demo(engine):
    prompts = [
        ("resonance", "analizar la frecuencia y resonancia de este patrón de pensamiento"),
        ("node",      "qué nodo φ gobierna la ética y la coherencia del sistema savant"),
        ("equation",  "muéstrame una ecuación del Hamiltoniano icosaédrico en el marco RRF"),
        ("chat",      "explícame la arquitectura savant engine y cómo evoluciona en las fases Φ"),
    ]

    for kind, text in prompts:
        print("=" * 80)
        print(f"[{kind.upper()}] prompt: {text}")
        try:
            reply = engine.respond(text)
        except Exception as e:
            reply = f"Error during respond(): {e}"
        print("→ response:")
        print(reply)
        print()

run_savant_demo(engine)


In [ ]:
# @title Optional interactive loop
ENABLE_LOOP = False  # pon True si quieres usar el loop interactivo

if ENABLE_LOOP:
    print("Interactive SavantEngine loop. Type 'exit' or 'salir' to quit.\n")
    while True:
        try:
            text = input("You: ").strip()
        except EOFError:
            break

        if text.lower() in {"exit", "quit", "salir"}:
            print("Exiting loop.")
            break

        if not text:
            continue

        try:
            reply = engine.respond(text)
        except Exception as e:
            reply = f"Error: {e}"
        print("SavantEngine:", reply)
        print()
else:
    print("Interactive loop disabled. Set ENABLE_LOOP = True to use it.")
